# Оценка метрик качества базовой модели

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import set_config

set_config(display='diagram')

In [ ]:
from titanic.data.make_dataset import load_titanic
from titanic.data.validation import train_test_split
from titanic.models.train import make_baseline_model

titanic = load_titanic()
train, test = train_test_split(titanic)
y_train = train['Survived']
y_test = test['Survived']

baseline_clf = make_baseline_model()
baseline_clf

## Обучение модели

In [ ]:
%%time
baseline_clf.fit(train, y_train);

## Оценка метрик

In [ ]:
from sklearn.metrics import plot_roc_curve

plot_roc_curve(baseline_clf, test, y_test);

In [ ]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test, baseline_clf.predict(test))

In [ ]:
import mlflow
import mlflow.sklearn

mlflow.set_tracking_uri("http://localhost:8081")

with mlflow.start_run(run_name="baseline-test") as run:
    sk_learn_rfr = baseline_clf

    # Log parameters and metrics using the MLflow APIs
    mlflow.log_metrics({"roc_auc": roc_auc_score(y_test, baseline_clf.predict(test))})

    # Log the sklearn model and register as version 1
#     mlflow.sklearn.log_model(
#         sk_model=sk_learn_rfr,
#         artifact_path="baseline",
#         registered_model_name="baseline-reg-model"
#     )